In [1]:
# Install Comet
!pip install comet_ml

In [2]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="J6p3oZezEzPg000PQdV3lEuTg",
    project_name="deviants",
    workspace="ronewa",
)

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install git+https://github.com/ProfHercules/Surprise

In [5]:
# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD,accuracy
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

In [6]:
sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/sample_submission.csv')
movies = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/movies.csv')
imdb = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/imdb_data.csv')
genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/genome_tags.csv')
train = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/train.csv')
test = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/test.csv')
tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/tags.csv')
links = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/links.csv')

In [8]:
train_subset = train
reader = Reader(rating_scale=(train_subset ['rating'].min(),train_subset['rating'].max()))
data = Dataset.load_from_df(train_subset[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = 0.01, random_state = 42)

In [ ]:
# SVD model

svd = SVD(n_epochs =40, n_factors = 2000, init_std_dev = 0.002, random_state=100,init_mean=0.001)

# train and test algorithm.
svd.fit(trainset)

predictions_svd = svd.test(testset)

# Compute and print Root Mean Squared Error
svd_rmse = accuracy.rmse(predictions_svd, verbose=True)

In [ ]:
params ={"random_state": 100,
         "model_type ": "SVD",
         "biased": True, 
         "verbose": True,
         'n_epochs': 40, 
         'n_factors': 2000, 
         'init_std_dev': 0.002, 

        }

best_metrics ={"RMSE" :svd_rmse}

In [ ]:
experiment.log_parameters(params)
experiment.log_metric('RMSE', r3)


In [ ]:
#STRICTLY FOR LOCAL JUPYTER NOTEBOOKS
experiment.end()

In [ ]:
# param_grid ={'n_epochs': [ 30, 45,50], 
#              'init_std_dev' : [0.002,0.005, 0.02], 
#              'n_factors' : [800,2000]}

# # Instatiate gridsearch instance
# gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=2)

# # Run gridsearch
# gs.fit(data)

# # best RMSE score
# print(gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params['rmse'])

In [ ]:
ratings=[]
for x,y in test.itertuples(index=False):
    output=svd.predict(x,y)
    ratings.append(output)
    
output_df=pd.DataFrame(ratings)[['uid','iid','est']]
output_df['ID']=output_df['uid'].astype(str) + '_' + output_df['iid'].astype(str)
output_df=output_df[['ID','est']]
output_df.head()

In [ ]:
#Creating the "results" dataframe and convert to csv
results = pd.DataFrame({"Id":output_df['ID'],"rating": output_df['est']})
results.to_csv("SVD.csv", index=False)